In [92]:
from selenium import webdriver
import time

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.webdriver.firefox.options import Options as FirefoxOptions
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import NoSuchElementException
# from selenium.webdriver.remote.webelement headless WebElement
from sys import platform
import random
import undetected_chromedriver as uc

import os
import chromedriver_autoinstaller
import pandas as pd
import json
import csv
from datetime import datetime
import re


In [95]:
delay = random.uniform(1, 3)
delay_short = random.uniform(0.5, 1)

# RECORD LIST OF LOCATION
results = []


list_target = [
    {
        "kota": "Jakarta Selatan",
        "kecamatan": "Cilandak",
        "lat": "-6.284528",
        "long": "106.800140",
    },
    # {
    #     "kota": "Jakarta Selatan",
    #     "kecamatan": "Jagakarsa",
    #     "lat": "-6.334917",
    #     "long": "106.823737",
    # },
]


for index, target in enumerate(list_target): 
    print(f"target : {target}") 

    # PAGE PARAMETERS
    keyword = "toko+sembako" 
    zoom = 16

    # OPEN PAGE
    url = f"https://www.google.com/maps/search/{keyword}/@{target['lat']},{target['long']},{zoom}z/data=!3m1!4b1?entry=ttu"
    options = uc.ChromeOptions()
    driver = uc.Chrome(options=options)
    wait = WebDriverWait(driver, 10)
    driver.get(url)
   
    time.sleep(delay)

    # # SCROLL DOWN REPETITION : no auto stop when data end
    # scrollable_list = driver.find_element(By.XPATH, "//div[contains(@role,'feed')]")
    # for index, value in enumerate(range(100)): # Adjust params for custom repetition
    #     print(index)
    #     scrollable_list.send_keys(Keys.PAGE_DOWN)
    #     time.sleep(0.5)  

    # SCROLL DOWN REPETITION : auto stop when data end

    scrollable_list = driver.find_element(By.XPATH, "//div[contains(@role,'feed')]")
    # max_scrolls = 100  # Maximum number of scrolls to perform
    max_scrolls = 1  # Maximum number of scrolls to perform
    scroll_count = 0
    end_condition_met = False
    while not end_condition_met and scroll_count < max_scrolls:
        try:
            driver.find_element(By.CSS_SELECTOR, '.HlvSq').text
            end_condition_met = True
            print("End of Page")
        except NoSuchElementException:
            scrollable_list.send_keys(Keys.PAGE_DOWN)
            time.sleep(0.5)  # Adjust this time as needed
            scroll_count += 1
            print(f"scroll_count: {scroll_count}")
    print("-" * 50)

    def get_element_text(navigator, selector): 
        try:
            element = wait.until(EC.visibility_of_element_located((navigator, selector)))
            return element.text
        except StaleElementReferenceException:
            # If StaleElementReferenceException occurs, try re-locating the element
            element = wait.until(EC.visibility_of_element_located((navigator, selector)))
            return element.text
        except Exception as error:
            print(f"An exception was thrown!:,{error}")
            return "" 

    # location_list = driver.find_elements(By.CSS_SELECTOR, '.hfpxzc') # list of location  
    location_list = driver.find_elements(By.XPATH, "//a[contains(@class,'hfpxzc')]")


    for index, location in enumerate(location_list):
        if index == 3:
            break
        
        time.sleep(delay_short) 
        location.click()

        result = {
            "title" : get_element_text(By.CSS_SELECTOR, '.DUwDvf.lfPIob' ),
            "phone" : get_element_text(By.XPATH, "//button[contains(@data-item-id,'phone')]"),
            "address" : get_element_text(By.CSS_SELECTOR, '.Io6YTe.fontBodyMedium.kR99db'),
            "rating" : get_element_text(By.CSS_SELECTOR, '.fontDisplayLarge'),
            # "reviewer" : get_element_text(By.CSS_SELECTOR, '.HHrUdb.fontTitleSmall.rqjGif').split()[0],
            "reviewer" : get_element_text(By.CSS_SELECTOR, '.HHrUdb.fontTitleSmall.rqjGif'),
            "detail" : get_element_text(By.CSS_SELECTOR, '.fontDisplayLarge'),
        }
        results.append(result)

        # try:
        #     location.click()
        # except ElementClickInterceptedException: # can't click handling
        #     print(location)
        #     print("-" * 50)
        #     continue 

        # try:
        #     # title = driver.find_element(By.CSS_SELECTOR, '.DUwDvf.lfPIob').text
        #     title = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, '.DUwDvf.lfPIob'))).text
        # except Exception as error:
        #     title = "" 
        #     print(f"An exception was thrown!:,{error}")
         
        # try:
        #     phone = driver.find_element(By.XPATH, "//button[contains(@data-item-id,'phone')]").text 
        # except Exception as error:
        #     phone = "" 
        #     print(f"An exception was thrown!:,{error}") 
        
        # try:
        #     address = driver.find_element(By.CSS_SELECTOR, '.Io6YTe.fontBodyMedium.kR99db').text
        # except Exception as error:
        #     address = "" 
        #     print(f"An exception was thrown!:,{error}")
                
        # try:
        #     rating = driver.find_element(By.CSS_SELECTOR, '.fontDisplayLarge').text
        # except Exception as error:
        #     rating = "" 
        #     print(f"An exception was thrown!:,{error}")
                
        # try:
        #     reviewer = driver.find_element(By.CSS_SELECTOR, '.HHrUdb.fontTitleSmall.rqjGif').text.split()[0]
        # except Exception as error:
        #     reviewer = ""
        #     print(f"An exception was thrown!:,{error}")
                
        # try:
        #     detail = driver.find_element(By.CSS_SELECTOR, '.fontDisplayLarge').text
        # except Exception as error:
        #     detail = ""
        #     print(f"An exception was thrown!:,{error}")

        # url = location.get_attribute('href')
        # match = re.search(r'!3d(-?\d+\.\d+)!4d(-?\d+\.\d+)', url)
        # latitude = ""
        # longitude = ""
        # if match:
        #     latitude = float(match.group(1))
        #     longitude = float(match.group(2))
                
            
        # results.append({
        #         "Store Name": title,
        #         "City": target["kota"],
        #         "Kecamatan": target["kecamatan"],
        #         "Address": address,
        #         "Latitude":latitude,
        #         "Longitude": longitude,
        #         "Rating": rating,
        #         "Reviewer": reviewer,
        #         "Phone": phone,
        #         "URL": url
        # })
        
        # scrollable_list.send_keys(Keys.PAGE_DOWN)

        
    # driver.quit()


# dt_now = datetime.now().strftime('%Y%m%d_%H_%M_%S')
# df_data = pd.DataFrame(results)
# df_data.to_csv(f'{keyword}{dt_now}.csv',index=False)
print("---DONE--")
print(results)

target : {'kota': 'Jakarta Selatan', 'kecamatan': 'Cilandak', 'lat': '-6.284528', 'long': '106.800140'}
scroll_count: 1
--------------------------------------------------
An exception was thrown!:,Message: 
Stacktrace:
	GetHandleVerifier [0x005072A3+45731]
	(No symbol) [0x00492D51]
	(No symbol) [0x0038880D]
	(No symbol) [0x003BB940]
	(No symbol) [0x003BBE0B]
	(No symbol) [0x003ED1F2]
	(No symbol) [0x003D8024]
	(No symbol) [0x003EB7A2]
	(No symbol) [0x003D7DD6]
	(No symbol) [0x003B31F6]
	(No symbol) [0x003B439D]
	GetHandleVerifier [0x00810716+3229462]
	GetHandleVerifier [0x008584C8+3523784]
	GetHandleVerifier [0x0085214C+3498316]
	GetHandleVerifier [0x00591680+611968]
	(No symbol) [0x0049CCCC]
	(No symbol) [0x00498DF8]
	(No symbol) [0x00498F1D]
	(No symbol) [0x0048B2C7]
	BaseThreadInitThunk [0x763F7BA9+25]
	RtlInitializeExceptionChain [0x773CBD3B+107]
	RtlClearBits [0x773CBCBF+191]

An exception was thrown!:,Message: 
Stacktrace:
	GetHandleVerifier [0x005072A3+45731]
	(No symbol) [0x004